In [ ]:
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit_machine_learning.algorithms import VQC
from qiskit_algorithms.optimizers import COBYLA
from qiskit_machine_learning.utils import algorithm_globals
import numpy as np
import time

# Agar hasil konsisten (reproducible)
algorithm_globals.random_seed = 42
np.random.seed(42)

print("--- 1. Persiapan Data ---")
# Load Data
iris = load_iris()
X = iris.data
y = iris.target

# PCA: Kompres 4 fitur -> 2 fitur (supaya muat di 2 qubit)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

# Scaling: Ubah ke range 0 s.d 1 (PENTING untuk rotasi sudut!)
scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler.fit_transform(X_pca)

# Bagi Data: 80% Latihan (Train), 20% Ujian (Test)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

print(f"Data Latihan: {len(X_train)} sampel")
print(f"Data Ujian: {len(X_test)} sampel")

# --- 2. Setup Komponen QML ---
num_qubits = 2

# Feature Map (Input Data)
feature_map = ZZFeatureMap(feature_dimension=num_qubits, reps=1)

# Ansatz (Model Otak)
# Kita pakai reps=2 supaya sirkuit lebih 'dalam' dan pintar menangani 3 kelas
ansatz = RealAmplitudes(num_qubits=num_qubits, reps=2)

# Optimizer
optimizer = COBYLA(maxiter=100) # Kita naikkan iterasi karena data lebih sulit

# --- 3. Rakit VQC ---
# VQC otomatis menangani logika multiclass di belakang layar
vqc = VQC(feature_map=feature_map,
          ansatz=ansatz,
          optimizer=optimizer)

# --- 4. Training ---
print("\n--- Mulai Training (Mungkin butuh 1-2 menit)... ---")
start_time = time.time()

# Fit model dengan data latihan
vqc.fit(X_train, y_train)

end_time = time.time()
print(f"Training Selesai dalam {end_time - start_time:.2f} detik!")

# --- 5. Evaluasi ---
# Cek seberapa pintar model kita memprediksi data ujian (yang belum pernah dilihatnya)
score = vqc.score(X_test, y_test)
print(f"\nAkurasi pada Data Test: {score:.2f} ({score*100}%)")

# Coba prediksi 3 sampel pertama dari data test
y_pred = vqc.predict(X_test[:3])
print(f"\nContoh Prediksi: {y_pred}")
print(f"Label Asli     : {y_test[:3]}")

/tmp/ipykernel_30760/1634778610.py:41: DeprecationWarning: The class ``qiskit.circuit.library.data_preparation._zz_feature_map.ZZFeatureMap`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. Use the zz_feature_map function as a replacement. Note that this will no longer return a BlueprintCircuit, but just a plain QuantumCircuit.
  feature_map = ZZFeatureMap(feature_dimension=num_qubits, reps=1)
/tmp/ipykernel_30760/1634778610.py:45: DeprecationWarning: The class ``qiskit.circuit.library.n_local.real_amplitudes.RealAmplitudes`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. Use the function qiskit.circuit.library.real_amplitudes instead.
  ansatz = RealAmplitudes(num_qubits=num_qubits, reps=2)
No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


--- 1. Persiapan Data ---
Data Latihan: 120 sampel
Data Ujian: 30 sampel

--- Mulai Training (Mungkin butuh 1-2 menit)... ---


QiskitMachineLearningError: 'Input data has incorrect shape, last dimension is not equal to the number of inputs: 2, but got: 4.'